In [1]:
!nvidia-smi

Fri Jan 27 23:52:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   40C    P0    53W / 300W |      0MiB / 32508MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2023-01-27 23:53:02.957375: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 23:53:03.130724: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3224009696458445910
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 32476168192
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9848476473341986004
physical_device_desc: "device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0"
xla_global_id: 416903419
]


2023-01-27 23:53:22.990867: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 23:53:27.170180: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0


In [7]:
import gc
import gdown
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
import tqdm.notebook as tqdm

In [8]:
aspect = ['biological','process']
aspect_abbr = 'bp'

In [9]:
file = 'cafa3_train/bp_df.csv'
# if not os.path.exists(file):
#     url = "https://drive.google.com/file/d/1RyeLQPFTMWAIr-OzELTWIx60ln-mZ7g_/view?usp=sharing"
#     output = file
#     gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [10]:
df = pd.read_csv(file)
df

,Entry,Sequence,Gene Ontology (biological process)
0,A0A060X6Z0,MPISSSSSSSTKSMRRAASELERSDSVTSPRFIGRRQSLIEDARKE...,GO:0008152;GO:0055114;GO:0044710;GO:0008150;GO...
1,A0A075F932,MVSESHHEALAAPPATTVAAAPPSNVTEPASPGGGGGKEDAFSKLK...,GO:0046883;GO:0009987;GO:0044763;GO:0044765;GO...
2,A0A078CGE6,MARQMTSSQFHKSKTLDNKYMLGDEIGKGAYGRVYIGLDLENGDFV...,GO:0009987;GO:0071704;GO:0043170;GO:0043412;GO...
3,A0A086F3E3,MTKGRLEAFSDGVLAIIITIMVLELKVPEGSSWASLQPILPRFLAY...,GO:0009987;GO:0044763;GO:0044765;GO:0098655;GO...
4,A0A087X1C5,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,GO:0009410;GO:0070887;GO:0009987;GO:0051716;GO...
...,...,...,...
53495,W5EP13,MLLFAPTPPPSPATAHRRPGGSAASCIRCSSVRELDRSPSRPPLPP...,GO:0009987;GO:0016311;GO:0008152;GO:0044237;GO...
53496,W8DXL4,MWLSACLCLVLSFLGGVNGTCPSQCSCEYHGRHDGSGSRLVLCNDL...,GO:0003008;GO:0050877;GO:0050953;GO:0007600;GO...
53497,W8E7I1,MSSEEGKLFVGGLNFNTDERALEDHFSSFGPISEVVVVKDRETQRS...,GO:0009987;GO:0051716;GO:0070417;GO:0009266;GO...
53498,X1WGX5,MEGKPRKKSFTPRDGKKPSFKSKGKPGGKPQGKRPFKPHNNDKGKG...,GO:0008354;GO:0009987;GO:0032504;GO:0044763;GO...


In [11]:
go_terms_bp = set()
for idx, row in df.iterrows():
    for term in row['Gene Ontology ('+' '.join(aspect)+')'].split(';'):
        go_terms_bp.add(term)
go_terms_bp = list(go_terms_bp)
go_terms_bp.sort()
print(len(go_terms_bp))
# print(go_terms_bp)

3992


In [12]:
def get_segments(sequence,segment_size=100,gap=30):
    segments = []
    start = 0
    end = segment_size
    while end <= len(sequence):
        segments.append(sequence[start:end])
        start += gap
        end += gap
    last_segment = sequence[start:]
    segments.append(last_segment)
    
    return segments

def get_training_data(df,segment_size=100,gap=30):
    training_data = list()
    for idx,row in tqdm.tqdm(df.iterrows()):
        labels = [0] * len(go_terms_bp)
        for term in row['Gene Ontology ('+' '.join(aspect)+')'].split(';'):
            labels[go_terms_bp.index(term)] = 1
        segments = get_segments(row['Sequence'],segment_size,gap)
        for segment in segments:
            training_data.append([row['Entry'],segment,labels])
    return training_data

In [13]:
training_data = get_training_data(df)
print(len(training_data))

0it [00:00, ?it/s]

878652


In [14]:
def get_ngrams(segment,n=3):
    ngrams = []
    for i in range(len(segment)-n+1):
        ngrams.append(segment[i:i+n])
    return ngrams

In [15]:
# Generate training data of ngrams
# if os.path.exists('bp/training_data_4grams.npy'):
#     print('Loading saved ngrams...')
#     training_data_ngrams = np.load('bp/training_data_4grams.npy',allow_pickle=True)
# else:
print('Preparing from scratch...')
training_data_ngrams = []

for i in tqdm.tqdm(range(len(training_data))):
    training_data_ngrams.append([training_data[i][0],get_ngrams(training_data[i][1],n=4),training_data[i][2]])
        
#     np.save('bp/training_data_4grams.npy',training_data_ngrams)
    
print(len(training_data_ngrams))

Preparing from scratch...


  0%|          | 0/878652 [00:00<?, ?it/s]

878652


In [16]:
def get_skip_grams(segment,skip=1,n=3):
    skip_grams = []
    window_size = skip + n
    for i in range(len(segment)-window_size+1):
        window = segment[i:i+window_size]
        indices = list(range(window_size))
        indices.pop(0)
        for idx in indices[::-1]:
            temp = ''
            for j in range(window_size):
                if j!=idx:
                    temp+=window[j]
            skip_grams.append(temp)

    return skip_grams

In [17]:
# if os.path.exists('bp/training_data_skip1_4grams.npy'):
#     print('Loading saved skip grams...')
#     training_data_skip_grams = np.load('bp/training_data_skip1_4grams.npy',allow_pickle=True)
# else:
print('Preparing from scratch...')
training_data_skip_grams = []
for i in tqdm.tqdm(range(len(training_data))):
    training_data_skip_grams.append([training_data[i][0],get_skip_grams(training_data[i][1],n=4),training_data[i][2]])
#     np.save('bp/training_data_skip1_4grams.npy',training_data_skip_grams)
print(len(training_data_skip_grams))

Preparing from scratch...


  0%|          | 0/878652 [00:00<?, ?it/s]

878652


In [18]:
import tensorflow as tf
import tensorflow_addons as tfa

In [19]:
#Assuming training_data as global variable

def train_test_split(X,y,fold_no,prev_index,Kfolds=5):
    test_split = 1/Kfolds
    
    start_index = prev_index
    end_index = (fold_no + 1) * (test_split) * len(X)
    end_index = round(end_index)
    
    if end_index==len(X):
        end_index -= 1
    
    entry = training_data[end_index][0]
    entries = [sample[0] for sample in training_data]
    
    first_occurence = entries.index(entry)
    entries.reverse()
    
    last_occurence = entries.index(entry)
    last_occurence = len(entries) - last_occurence - 1
    
    del entries
    gc.collect()
    
    end_index = first_occurence if (abs(end_index-first_occurence) < abs(end_index-last_occurence)) else last_occurence
    
    X_test = X[start_index:end_index+1]
    y_test = y[start_index:end_index+1]
    X_train = X[:start_index]
    X_train.extend(X[end_index+1:])
    y_train = y[:start_index]
    y_train.extend(y[end_index+1:])
    
    return X_train, y_train, X_test, y_test, start_index, end_index + 1

In [20]:
MAX_WORDS = 331776
MAX_LEN_NG = 97 #100
MAX_LEN_SG = 384 #300

def tokenization(X_train,X_test,maxlen):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_WORDS)
    tokenizer.fit_on_texts(X_train)

    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

    vocab_size = len(tokenizer.word_index) + 1

    X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=maxlen)
    
    return X_train, X_test, vocab_size, tokenizer

In [21]:
from tensorflow.keras import backend as K

false_negative_penalty = 6
false_positive_penalty = 1

def custom_loss(y_true, y_logit):

    loss = float(0)
    y_true = tf.cast(y_true, tf.float32)
    y_logit = tf.cast(y_logit, tf.float32)
    
    first_term = false_negative_penalty * float(y_true) * - K.log(y_logit + K.epsilon())
    second_term = false_positive_penalty * (1 - float(y_true)) * - K.log(1 - y_logit + K.epsilon())
    
    loss = K.mean(first_term+second_term)

    return loss

def precision(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=1)
    precision = true_positives / (predicted_positives + K.epsilon())
    
    return K.mean(precision)


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=1)
    recall = true_positives / (possible_positives + K.epsilon())
    return K.mean(recall)

def f1_score(y_true, y_pred):
    rec = recall(y_true,y_pred)
    prec = precision(y_true,y_pred)
    f1 = 2*prec*rec/(prec+rec)
    return f1

In [22]:
class attention(tf.keras.layers.Layer):
    
    def __init__(self, return_sequences=True,**kwargs):
        self.return_sequences = return_sequences
        super(attention,self).__init__()

    def get_config(self):
      config = super().get_config().copy()
      config.update({
          'return_sequences': self.return_sequences 
      })
      return config
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [23]:
NUM_CLASSES = 3992 #For bp (Change according to aspects)

def get_model_ng_sg(vocab_size_ng, vocab_size_sg):
    #Input layers

    input_ngrams = tf.keras.layers.Input(shape=(MAX_LEN_NG,)) 
    input_skip_grams = tf.keras.layers.Input(shape=(MAX_LEN_SG,)) 

    #embeddings
    embedding_layer_ngrams = tf.keras.layers.Embedding(vocab_size_ng, 32)(input_ngrams)
    embedding_layer_skip_grams = tf.keras.layers.Embedding(vocab_size_sg, 32)(input_skip_grams)

    #BI-LSTMs for each of the inputs
    sequence_output_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70,return_sequences=True))(embedding_layer_ngrams)
    attention_output_1 = attention(return_sequences=False)(sequence_output_1)
    dropout_1 = tf.keras.layers.Dropout(0.3)(attention_output_1)
    dense_layer_1 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout_1)

    sequence_output_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70, return_sequences=True))(embedding_layer_skip_grams)
    attention_output_2 = attention(return_sequences=False)(sequence_output_2)
    dropout_2 = tf.keras.layers.Dropout(0.3)(attention_output_2)
    dense_layer_2 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout_2)

    max_layer = tf.keras.layers.Maximum()([dense_layer_1,dense_layer_2])

    model = tf.keras.models.Model(
        inputs=[
            input_ngrams,
            input_skip_grams
        ], 
        outputs=max_layer)
    
    return model

In [24]:
def evaluate_annotations(real_annots, pred_annots):
    total = 0
    p = 0.0
    r = 0.0
    p_total= 0
    fps = []
    fns = []
    for i in range(len(real_annots)):
        if len(real_annots[i]) == 0:
            continue
        tp = set(real_annots[i]).intersection(set(pred_annots[i]))
        fp = set(pred_annots[i]) - tp
        fn = set(real_annots[i]) - tp
        
        fps.append(fp)
        fns.append(fn)
        tpn = len(tp)
        fpn = len(fp)
        fnn = len(fn)
        total += 1
        recall = tpn / (1.0 * (tpn + fnn))
        r += recall
        if len(pred_annots[i]) > 0:
            p_total += 1
            precision = tpn / (1.0 * (tpn + fpn))
            p += precision
    r /= total
    if p_total > 0:
        p /= p_total
    f = 0.0
    if p + r > 0:
        f = 2 * p * r / (p + r)
    return f, p, r, fps, fns

### Model using skip grams and ngrams

In [25]:
#Considering ngrams and skip grams
X_train_ng = [' '.join(sample[1]) for sample in training_data_ngrams]
X_train_sg = [' '.join(sample[1]) for sample in training_data_skip_grams]
y_train = [sample[2] for sample in training_data]

del training_data_ngrams
del training_data_skip_grams

In [26]:
print(len(X_train_ng),len(X_train_sg))

878652 878652


In [27]:
test_df = pd.read_csv('cafa3_test/bp_df.csv')
test_df

,Entry,Sequence,Gene Ontology (biological process)
0,T100900000026,MAESFKELDPDSSMGKALEMTCAIQNQLARILAEFEMTLERDVLQP...,GO:0019222;GO:0009987;GO:0044763;GO:0008150;GO...
1,T100900000115,MNNLSFSELCCLFCCPPCPGKIASKLAFLPPDPTYTLMCDESGSRW...,GO:0044380;GO:0071704;GO:0043170;GO:0043412;GO...
2,T100900000116,MPEPGPRMNGFSLGELCWLFCCPPCPSRIAAKLAFLPPEPTYTVLA...,GO:0044380;GO:0071704;GO:0009057;GO:0043170;GO...
3,T100900000141,MAVPGPTARAGARPRLDLQLVQRFVRIQKVFFPSWSSQNVLMFMTL...,GO:0010033;GO:0009987;GO:0070887;GO:0051716;GO...
4,T100900000161,MADDLEQQPQGWLSSWLPTWRPTSMSQLKNVEARILQCLQNKFLAR...,GO:0009987;GO:0071704;GO:0044763;GO:0009308;GO...
...,...,...,...
2387,T992870000466,MIPEKRIIRRIQSGGCAIHCQDCSISQLCIPFTLNEHELDQLDNII...,GO:0034645;GO:0097659;GO:0016070;GO:0071704;GO...
2388,T992870000685,MPEVKTEKPHLLDMGKPQLRMVDLNLLTVFDAVMQEQNITRAAHTL...,GO:0034645;GO:0097659;GO:0016070;GO:0071704;GO...
2389,T992870001023,MMRVLVVEDNALLRHHLKVQLQDSGHQVDAAEDAREADYYLNEHLP...,GO:0034645;GO:0097659;GO:0016070;GO:0071704;GO...
2390,T992870001087,MIDGKTANEIFDSIRQHIIAGTLRAEDSLPPVRELASELKVNRNTV...,GO:0034645;GO:0097659;GO:0051253;GO:0071704;GO...


In [28]:
testing_data = get_training_data(test_df)
print(len(testing_data))

0it [00:00, ?it/s]

35365


In [29]:
print('Preparing from scratch...')
testing_data_ngrams = []

for i in tqdm.tqdm(range(len(testing_data))):
    testing_data_ngrams.append([testing_data[i][0],get_ngrams(testing_data[i][1],n=4),testing_data[i][2]])
    
print(len(testing_data_ngrams))

Preparing from scratch...


  0%|          | 0/35365 [00:00<?, ?it/s]

35365


In [30]:
print('Preparing from scratch...')
testing_data_skip_grams = []

for i in tqdm.tqdm(range(len(testing_data))):
    testing_data_skip_grams.append([testing_data[i][0],get_skip_grams(testing_data[i][1],n=4),testing_data[i][2]])
    
print(len(testing_data_skip_grams))

Preparing from scratch...


  0%|          | 0/35365 [00:00<?, ?it/s]

35365


In [31]:
#Considering ngrams and skip grams
X_test_ng = [' '.join(sample[1]) for sample in testing_data_ngrams]
X_test_sg = [' '.join(sample[1]) for sample in testing_data_skip_grams]
y_test = [sample[2] for sample in testing_data]

del testing_data_ngrams
del testing_data_skip_grams

In [32]:
print(len(X_test_ng),len(X_test_sg))

35365 35365


In [33]:
def compute_metrics(predictions):
    final_predictions = []
    actual_y_test = []

    current_entry = ''
    counter = 0
    total_counts = 0
    start_index = 0

    if len(predictions) == len(testing_data):
        temp = np.zeros(NUM_CLASSES)
        for i in range(len(predictions)):
            if current_entry != testing_data[start_index+i][0]:
                #compute prev
                if i!=0:
                    temp /= counter
                    final_predictions.append(temp)

                #reset
                total_counts += counter
                counter = 1
                temp = np.zeros(NUM_CLASSES)

                #init new
                current_entry = testing_data[start_index+i][0]
                temp += np.array(predictions[i])
                actual_y_test.append(testing_data[start_index+i][2])
            else:
                temp += np.array(predictions[i])
                counter += 1

        total_counts += counter
        temp /= counter
        final_predictions.append(temp)

    else:
        print('Lengths of predictions dont match with test data')
    
    final_predictions = np.array(final_predictions, dtype=float)
    actual_y_test = np.array(actual_y_test, dtype=float)
    
    #Computing maximal F1-score
    fmax = 0.0
    tmax = 0.0
    precisions = []
    recalls = []
    
    for t in tqdm.tqdm(range(0, 101)):
        
        threshold = t / 100.0
        pred_annots = []
        real_annots = []
    
        for i in range(len(final_predictions)):
            new_preds = []
            new_ys = []
            for j in range(NUM_CLASSES):
                if final_predictions[i][j]>=threshold:
                    new_preds.append(go_terms_bp[j]) #GO_TERMS_BP
                if actual_y_test[i][j]==1:
                    new_ys.append(go_terms_bp[j])
            pred_annots.append(new_preds)
            real_annots.append(new_ys)
    
        fscore, prec, rec, fps, fns = evaluate_annotations(real_annots, pred_annots)
        avg_fp = sum(map(lambda x: len(x), fps)) / len(fps)
        print(f'{avg_fp}')
        precisions.append(prec)
        recalls.append(rec)
        print(f'Fscore: {fscore}, Precision: {prec}, Recall: {rec}, threshold: {threshold}')
        if fmax < fscore:
            fmax = fscore
            tmax = threshold
    print('\nFinal Results (Maximal F1-score):')
    print(f'Fmax: {fmax:0.3f}, threshold: {tmax}')
    precisions = np.array(precisions)
    recalls = np.array(recalls)
        
    
#     rec = recall(actual_y_test,final_predictions)
#     prec = precision(actual_y_test,final_predictions)
#     f1 = f1_score(actual_y_test,final_predictions)
    
#     return rec,prec,f1

In [34]:

recs = []
precs = []
f1s = []

for i in range(1):
    
#     print('Splitting into train-test...')
#     X_train_ng, y_train, X_test_ng, y_test, start_index, prev_index1 = train_test_split(X_ng,y,i,prev_index,Kfolds)
#     X_train_sg, _, X_test_sg, _, _, _= train_test_split(X_sg,y,i,prev_index,Kfolds)
    
#     prev_index = prev_index1
    
    print('Tokenizing...')
    X_train_ng, X_test_ng, vocab_size_ng, tokenizer1 = tokenization(X_train_ng, X_test_ng, MAX_LEN_NG)  
    X_train_sg, X_test_sg, vocab_size_sg, tokenizer2 = tokenization(X_train_sg, X_test_sg, MAX_LEN_SG)
    
   
    print('Shuffling...')   
    shuffled = [[X_train_ng[i],X_train_sg[i],y_train[i]] for i in range(len(X_train_ng))]
    np.random.shuffle(shuffled)

    X_train_ng = [shuffled[i][0] for i in range(len(shuffled))]
    X_train_sg = [shuffled[i][1] for i in range(len(shuffled))]
    y_train = [shuffled[i][2] for i in range(len(shuffled))]
    X_train_ng = np.array(X_train_ng)
    X_train_sg = np.array(X_train_sg)
    y_train = np.array(y_train)
    
    
    model = get_model_ng_sg(vocab_size_ng, vocab_size_sg)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])

    print('Training...')
    history = model.fit([X_train_ng,X_train_sg], y_train, batch_size=32, epochs=5,validation_split=0.2)
    
    print('Evaluating model...')
    predictions = model.predict([X_test_ng,X_test_sg])
    
    print('Computing Metrics...\n')
    compute_metrics(predictions)
    
#     recs.append(rec.numpy())
#     precs.append(prec.numpy())
#     f1s.append(f1.numpy())

# print('Recall:',sum(recs)/len(recs))
# print('Precision:',sum(precs)/len(precs))
# print('F1-Score:',sum(f1s)/len(f1s))

Tokenizing...
Shuffling...


2023-01-28 00:22:49.510943: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0


Training...


2023-01-28 00:22:51.857638: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 22448485056 exceeds 10% of free system memory.
2023-01-28 00:23:07.873143: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 22448485056 exceeds 10% of free system memory.


Epoch 1/5


2023-01-28 00:23:27.721160: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


21967/21967 [==============================] - 1014s 46ms/step - loss: 0.1834 - recall: 0.4919 - precision: 0.2888 - f1_score: 0.3619 - val_loss: 0.1683 - val_recall: 0.4967 - val_precision: 0.3072 - val_f1_score: 0.3781
Epoch 2/5
21967/21967 [==============================] - 1004s 46ms/step - loss: 0.1562 - recall: 0.5157 - precision: 0.3277 - f1_score: 0.3991 - val_loss: 0.1522 - val_recall: 0.5239 - val_precision: 0.3466 - val_f1_score: 0.4158
Epoch 3/5
21967/21967 [==============================] - 1004s 46ms/step - loss: 0.1376 - recall: 0.5522 - precision: 0.3636 - f1_score: 0.4371 - val_loss: 0.1400 - val_recall: 0.5645 - val_precision: 0.3577 - val_f1_score: 0.4366
Epoch 4/5
21967/21967 [==============================] - 1005s 46ms/step - loss: 0.1224 - recall: 0.5952 - precision: 0.3893 - f1_score: 0.4695 - val_loss: 0.1312 - val_recall: 0.5659 - val_precision: 0.4038 - val_f1_score: 0.4700
Epoch 5/5
21967/21967 [==============================] - 1004s 46ms/step - loss: 0.110

  0%|          | 0/101 [00:00<?, ?it/s]

3962.0727424749166
Fscore: 0.014882048137041373, Precision: 0.007496807997265417, Recall: 1.0, threshold: 0.0
1316.0773411371238
Fscore: 0.043628233353920264, Precision: 0.02232255182489113, Recall: 0.9577362069031286, threshold: 0.01
875.1321070234113
Fscore: 0.0626085275606236, Precision: 0.03239180667706394, Recall: 0.9323785420951314, threshold: 0.02
668.813127090301
Fscore: 0.07859257982845803, Precision: 0.04106380837972357, Recall: 0.9129496243313415, threshold: 0.03
544.9142976588629
Fscore: 0.09299499415867242, Precision: 0.04904033692656435, Recall: 0.8967347768669801, threshold: 0.04
461.21488294314383
Fscore: 0.10628077584692557, Precision: 0.05654112799702743, Recall: 0.8835187078096112, threshold: 0.05
400.06103678929765
Fscore: 0.11867380564409015, Precision: 0.06366909403746152, Recall: 0.8720591169255077, threshold: 0.06
353.34740802675583
Fscore: 0.13024828182817907, Precision: 0.07045764103434778, Recall: 0.8603155975879196, threshold: 0.07
315.73327759197326
Fscore:

13.32901337792642
Fscore: 0.4608133116210171, Precision: 0.49015909023632037, Recall: 0.4347829002878031, threshold: 0.67
12.640886287625419
Fscore: 0.45954489442282515, Precision: 0.49882990793077886, Recall: 0.4259958760520955, threshold: 0.68
11.993729096989966
Fscore: 0.45866483735287855, Precision: 0.5072927193609819, Recall: 0.41854418066713406, threshold: 0.69
11.320234113712374
Fscore: 0.4579981382549503, Precision: 0.5177711063408056, Recall: 0.41059758596208695, threshold: 0.7
10.681438127090301
Fscore: 0.45650203000741957, Precision: 0.5273549895204155, Recall: 0.40243300618054023, threshold: 0.71
10.076505016722408
Fscore: 0.45432705001522167, Precision: 0.5363663699234266, Recall: 0.39405483258252344, threshold: 0.72
9.507107023411372
Fscore: 0.4517489438523788, Precision: 0.5456597519431727, Recall: 0.38541676563409666, threshold: 0.73
8.92767558528428
Fscore: 0.4491919984267938, Precision: 0.556257768241177, Recall: 0.3766887864240899, threshold: 0.74
8.389214046822742
F

In [35]:
model.save('bp_model.h5')
np.save('bp_predictions',predictions)

In [36]:
history_new = model.fit([X_train_ng,X_train_sg], y_train, batch_size=32, epochs=5,validation_split=0.2)
predictions = model.predict([X_test_ng,X_test_sg])
compute_metrics(predictions)

2023-01-28 01:57:05.395867: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 22448485056 exceeds 10% of free system memory.
2023-01-28 01:57:21.667675: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 22448485056 exceeds 10% of free system memory.


Epoch 1/5
21967/21967 [==============================] - 1009s 46ms/step - loss: 0.1014 - recall: 0.6667 - precision: 0.4310 - f1_score: 0.5225 - val_loss: 0.1184 - val_recall: 0.6201 - val_precision: 0.4257 - val_f1_score: 0.5036
Epoch 2/5
21967/21967 [==============================] - 1003s 46ms/step - loss: 0.0941 - recall: 0.6941 - precision: 0.4486 - f1_score: 0.5440 - val_loss: 0.1149 - val_recall: 0.6255 - val_precision: 0.4533 - val_f1_score: 0.5244
Epoch 3/5
21967/21967 [==============================] - 1002s 46ms/step - loss: 0.0881 - recall: 0.7160 - precision: 0.4645 - f1_score: 0.5625 - val_loss: 0.1112 - val_recall: 0.6376 - val_precision: 0.4680 - val_f1_score: 0.5386
Epoch 4/5
21967/21967 [==============================] - 1060s 48ms/step - loss: 0.0832 - recall: 0.7351 - precision: 0.4784 - f1_score: 0.5787 - val_loss: 0.1085 - val_recall: 0.6549 - val_precision: 0.4684 - val_f1_score: 0.5450
Epoch 5/5
1106/1106 [==============================] - 20s 18ms/step


  0%|          | 0/101 [00:00<?, ?it/s]

3962.0727424749166
Fscore: 0.014882048137041373, Precision: 0.007496807997265417, Recall: 1.0, threshold: 0.0
821.6404682274248
Fscore: 0.06909580260718964, Precision: 0.03589061048541132, Recall: 0.9234652488920075, threshold: 0.01
548.9531772575251
Fscore: 0.09636716987215138, Precision: 0.050934419423618774, Recall: 0.8921666990803299, threshold: 0.02
423.77801003344484
Fscore: 0.11786802433569563, Precision: 0.06321806994748509, Recall: 0.8696648586818043, threshold: 0.03
348.48745819397993
Fscore: 0.13683357158418213, Precision: 0.07436830655508037, Recall: 0.8549143491089671, threshold: 0.04
297.41471571906357
Fscore: 0.15353770013894277, Precision: 0.08447561184185161, Recall: 0.8414812565318283, threshold: 0.05
259.810618729097
Fscore: 0.16871928323694907, Precision: 0.09392145428560097, Recall: 0.8286274265819035, threshold: 0.06
230.55016722408027
Fscore: 0.18304197060107918, Precision: 0.10306414146737464, Recall: 0.8171536137879851, threshold: 0.07
207.37123745819397
Fscore

10.206939799331103
Fscore: 0.47164442834973913, Precision: 0.5671392188223006, Recall: 0.40367389675758925, threshold: 0.68
9.747909698996656
Fscore: 0.4703636481940246, Precision: 0.5760990185481615, Recall: 0.39742207271411045, threshold: 0.69
9.292642140468228
Fscore: 0.46885365019686626, Precision: 0.5846196882360067, Recall: 0.3913573114405887, threshold: 0.7
8.85994983277592
Fscore: 0.466374492457912, Precision: 0.592567155882742, Recall: 0.3844931215524485, threshold: 0.71
8.445234113712374
Fscore: 0.4634502093159597, Precision: 0.6012534509062267, Recall: 0.37703607923928767, threshold: 0.72
8.069816053511706
Fscore: 0.46021289758281964, Precision: 0.6091887175783355, Recall: 0.3697831948044375, threshold: 0.73
7.676839464882943
Fscore: 0.4565085217641863, Precision: 0.6172212643606948, Recall: 0.3621988041078448, threshold: 0.74
7.273411371237458
Fscore: 0.45322558061754364, Precision: 0.6261016458266203, Recall: 0.3551604482026552, threshold: 0.75
6.915969899665551
Fscore: 0.

In [37]:
model.save('bp_model_10epoch.h5')
np.save('bp_predictions_10epoch',predictions)